# Energie di stato fondamentale con algoritmi classici

Seguo l'esempio di Rodolfo in example_classic_script.py, facendo qualche variazione per prendere la mano.

Uso gli algoritmi implementati nella libreria pyscf.

In [15]:
import numpy as np
from pyscf import gto,scf,ao2mo,mp,cc,fci,tools 
import matplotlib.pyplot as plt 

## 1 - Implementare una molecola

Servono:
- elemento (lettera)
- posizione (coordinate cartesiane)
- carica complessiva 
- spin totale degli elettroni
- base

L'idea è trovare la _distanza di legame_, quindi quella distanza interatomica $d$ che minimizza l'energia di stato fondamentale. Per farlo si calcola l'energia (con diversi algoritmi) a molteplici distanze (bisogna dichiarare una molecola per ogni distanza), quindi si plottano i risultati.

Magari poi lo eseguo con basi diverse.

In [16]:
# dichiaro l'array delle distanze
arr_d = np.arange(0.3, 4, .05) # da 0.3 a 3.95 (Angstrom) con passo 0.05
# array con le altezze (triangolo equilatero)
arr_h = np.sqrt(arr_d**2 - (arr_d/2)**2) # **2 = al quadrato


In [17]:
# Sarebbe bello fare un'iterazione che esegue tutti i metodi diversi per delle basi
arr_basis = ['6-31g', 'sto-6g', '6-31g', 'cc-pvdz', 'aug-cc-pvdz']

D'ora in poi devo fare tutto in un blocco codice, altrimenti ogni volta ripeto la formula «for basis in arr_basis:»

Non so quale sia la cosa più 'pythonica', ma forse più la seconda

In [18]:
# Creo un array per i dizionari
arr_energies = []

# Per ciascuna base
for basis in arr_basis:
    # Creo il dizionario con le energie
    energies = {
    "Basis": basis,
    "HF": [],
    "FCI": [],
    "MP2": [],
    "CCSD": []
    }
    # Aggiungo il dizionario all'array
    arr_energies.append(energies)

Poi sarà interessante provare con molecole diverse. Per ora provo un ossigeno biatomico.

In [19]:
# Devo fare un array pure di mol altrimenti non posso richiamarlo dopo
# Questi notebook hanno dei limiti   

arr_molecules = []

# Per ciascuna base
for basis in arr_basis:

    molecules = {
    'Basis': basis,
    'Mols': [],
    "CampoMedio": []
    }

    # Per ciascuna distanza
    for d in arr_d:
        # Inizializzo la molecola (per ora O_2 lineare a distanza d)
        geometry = "O .0 .0 .0; O .0 .0 " + str(d)

        mol = gto.M(
            atom=geometry,
            charge=0,
            spin=0,
            basis=basis,
            symmetry=True, # (?) da approfondire
            verbose=0 # (?)
        )
    
        molecules['Mols'].append(mol)
    arr_molecules.append(molecules)

### 1.1 Campo medio (Hartree-Fock)
Se gli elettroni con spin up sono tanti quanti gli elettroni con spin down (spin totale 0) si fa un Restricted Hartree Fock. I metodi successivi prenderanno come argomento il campo medio di HF

Fonti consigliate su RHF, ROHF, UHF 

- Szabo, Ostlund - Modern Quantum Chemistry 
- https://en.wikipedia.org/wiki/Hartree%E2%80%93Fock_method
- https://en.wikipedia.org/wiki/Restricted_open-shell_Hartree%E2%80%93Fock
- https://en.wikipedia.org/wiki/Unrestricted_Hartree%E2%80%93Fock

In [20]:
# Per ciascuna base
for i in range (len(arr_basis)):
    # Per ciascuna distanza
    for j in range (len(arr_d)):
        # campo medio con restricted HF
        cm  = scf.RHF(arr_molecules[i]['Mols'][j])
        e_HF = cm.kernel() # chiamando il kernel otteniamo l'energia calcolata dal metodo
        
        arr_energies[i]['HF'].append(e_HF)
        arr_molecules[i]['CampoMedio'].append(cm)
	    

### 1.2 FCI 
Quando il sistema ha pochi elettroni, possiamo permetterci di diagonalizzare esattamente l' Hamiltoniano del sistema nella base degli orbitali scelta.

In [21]:
# Supera ampiamente i 6 minuti di esecuzione
# Probabilmente con alcune basi diventa estremamente impegnativo

# Per ciascuna base
for i in range (len(arr_basis)):
    # Per ciascuna distanza
    for j in range (len(arr_d)):
        cm = arr_molecules[i]['CampoMedio'][j]
        fci_calc = fci.FCI(cm) # <- nei metodi correlati passiamo come argomento un conto di campo medio, HF
        e_fci = fci_calc.kernel()[0]

        arr_energies[i]['FCI'].append(e_fci)

# questo pezzo di codice non può essere eseguito su tutte le basi

FCI diventa particolarmente oneroso quando incrementiamo la taglia della base o il numero di elettroni, perciò si ricorre a approssimazioni.


### 1.3 MP2 

Possiamo ad esempio aggiungere perturbativamente le interazioni tra gli elettroni. Questa tecnica prende il nome di Møller-Plesset. Nel nostro caso consideriamo una perturbazione al secondo ordine.


In [2]:
# Per ciascuna base
for i in range (len(arr_basis)):
    # Per ciascuna distanza
    for j in range (len(arr_d)): # è impazzito il notebook: non li uso mai più
		cm = arr_molecules[i]['CampoMedio'][j]
		mp2   = mp.MP2(cm)
		e_mp2 = mp2.kernel()[0]
		e_mp2 += Ehf # <- questa linea è necessaria perché MP2 generalmente mostra l'energia di differenza con HF
		arr_energies[i]['MP2'].append(e_mp2)

TabError: inconsistent use of tabs and spaces in indentation (<string>, line 5)